<a href="https://colab.research.google.com/github/Rhqo/lora/blob/master/Korean_LoRA_Finetuning_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🇰🇷 한국어 LoRA Fine-tuning with TinyLlama

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/Korean_LoRA_Finetuning_Colab.ipynb)

---

## 📌 이 노트북의 목적

영어 기반 TinyLlama 모델을 **LoRA(Low-Rank Adaptation)**를 사용하여 한국어 데이터로 fine-tuning합니다.

### 🎯 학습 효과
- **Before**: 한국어 질문에 영어로 답변하거나 이상한 답변
- **After**: 자연스러운 한국어로 정확한 답변!

### ⏱️ 예상 소요 시간
- 설치: ~3분
- 학습: ~15-20분 (T4 GPU 기준)

---

## ⚠️ 시작하기 전에

### 1. GPU 설정 확인
상단 메뉴: **런타임(Runtime) > 런타임 유형 변경(Change runtime type) > 하드웨어 가속기(Hardware accelerator) > GPU > T4**

### 2. 노트북 실행 방법
- 각 셀을 순서대로 실행 (Shift + Enter)
- 또는 상단 메뉴: **런타임 > 모두 실행**

## 1️⃣ 환경 설정

In [ ]:
# GPU 확인
!nvidia-smi

Thu Oct 16 16:07:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# 필요한 패키지 설치 (~3분 소요)
print("📦 Installing packages...")

!pip install -q -U \
    torch>=2.2.0 \
    transformers==4.41.2 \
    datasets==2.18.0 \
    accelerate==0.31.0 \
    peft==0.11.1 \
    trl==0.9.4 \
    bitsandbytes==0.43.1 \
    sentencepiece==0.2.0 \
    protobuf

print("✅ Installation complete!")

📦 Installing packages...
✅ Installation complete!


In [ ]:
# 라이브러리 임포트
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM, prepare_model_for_kbit_training
from trl import SFTTrainer
import os
from datetime import datetime

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

✅ PyTorch version: 2.9.0+cu128
✅ CUDA available: True
✅ GPU: Tesla T4


## 2️⃣ 설정 및 하이퍼파라미터

In [ ]:
# ====== 이 부분을 수정하여 실험할 수 있습니다 ======

# 모델 설정
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
OUTPUT_DIR = "./korean-tinyllama-lora"

# 데이터 설정
DATASET_NAME = "nlpai-lab/kullm-v2"
NUM_SAMPLES = 3000

# LoRA 하이퍼파라미터
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj"]

# 학습 하이퍼파라미터
NUM_EPOCHS = 3
BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 5e-5
MAX_SEQ_LENGTH = 512

# 4-bit Quantization 설정 (메모리 절약)
USE_4BIT = True  # T4 GPU에서는 True 권장

# ===============================================

print("📋 Configuration:")
print(f"   Model: {MODEL_NAME}")
print(f"   Dataset: {DATASET_NAME} ({NUM_SAMPLES} samples)")
print(f"   LoRA Rank: {LORA_R}")
print(f"   Epochs: {NUM_EPOCHS}")
print(f"   Effective Batch Size: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"   4-bit Quantization: {USE_4BIT}")

📋 Configuration:
   Model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
   Dataset: nlpai-lab/kullm-v2 (3000 samples)
   LoRA Rank: 16
   Epochs: 3
   Effective Batch Size: 16
   4-bit Quantization: True


## 3️⃣ 데이터셋 준비

In [ ]:
print("📚 Loading Korean dataset...")

# 한국어 instruction 데이터셋 로드
try:
    dataset = load_dataset(DATASET_NAME, split="train")
    print(f"✅ Loaded {DATASET_NAME}")
except Exception as e:
    print(f"⚠️  Failed to load {DATASET_NAME}, trying alternative...")
    # Note: The DATASET_NAME variable is already set in the hyperparameters cell.
    # This fallback logic might be redundant if the hyperparameters cell is the single source of truth.
    # Keeping it for now, but consider removing if it causes confusion.
    dataset = load_dataset("nlpai-lab/kullm-v2", split="train") # Directly use the intended dataset name
    print(f"✅ Loaded nlpai-lab/kullm-v2")


# 샘플링
dataset = dataset.shuffle(seed=42).select(range(min(NUM_SAMPLES, len(dataset))))
print(f"📊 Selected {len(dataset)} samples")

# 데이터 포맷팅
def format_prompt(examples):
    """ChatML 형식으로 포맷팅 (Batched version)"""
    texts = []
    for instruction, output in zip(examples.get("instruction", []), examples.get("output", [])):
         # 빈 데이터 필터링
        if instruction and output:
            text = f"<|user|>\n{instruction}</s>\n<|assistant|>\n{output}</s>"
            texts.append(text)
        else:
            texts.append("") # 빈 문자열 추가 또는 해당 샘플 건너뛰기 (여기서는 빈 문자열로 처리)

    return {"text": texts}


# 포맷 적용 및 필터링
# Use batched=True for potentially faster processing
# Remove columns after mapping
dataset = dataset.map(format_prompt, remove_columns=dataset.column_names, batched=True)
# Filter out empty strings resulting from invalid data
dataset = dataset.filter(lambda x: len(x.get("text", "")) > 10)


print(f"✅ Formatted {len(dataset)} samples")
print(f"\n📝 Example data:")
print("=" * 60)
# Ensure dataset is not empty before accessing elements
if len(dataset) > 0:
    print(dataset[0]["text"][:300] + "...")
else:
    print("No data left after filtering.")
print("=" * 60)

📚 Loading Korean dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Loaded nlpai-lab/kullm-v2
📊 Selected 3000 samples
✅ Formatted 2978 samples

📝 Example data:
<|user|>
아래 파이썬 프로그램에서 스크랩한 URL이 뉴스 기사인지 확인하기 위해 5개의 검사를 추가합니다. URL에 뉴스 또는 보도 자료가 포함되어 있는지 확인하여 통과된\_url에 추가하고, 그렇지 않으면 실패한\_url에 추가합니다.CSS 선택기에 뉴스 또는 보도 자료가 포함되어 있는지 확인하여 통과 된 \_url에 추가하지 않으면 실패한 \_url에 추가합니다.링크가 html인지 확인하여 passed\_url에 추가합니다.URL이 pdf 또는 미디어 파일 확장자 또는 정적 파일 확장자인지 확인한 후 failed\_url...


## 4️⃣ 모델 로딩

In [ ]:
print("⬆️ Upgrading bitsandbytes...")
!pip install -qU bitsandbytes
print("✅ bitsandbytes upgraded!")

⬆️ Upgrading bitsandbytes...
✅ bitsandbytes upgraded!


In [ ]:
print("🤖 Loading model and tokenizer...")

# 4-bit quantization 설정 (옵션)
if USE_4BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )
    print("   Using 4-bit quantization")
else:
    bnb_config = None
    print("   Using full precision")

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config if USE_4BIT else None,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16 if not USE_4BIT else None,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# 4-bit 모델 준비
if USE_4BIT:
    model = prepare_model_for_kbit_training(model)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✅ Model and tokenizer loaded")
print(f"   Total parameters: {model.num_parameters() / 1e9:.2f}B")

🤖 Loading model and tokenizer...
   Using 4-bit quantization
✅ Model and tokenizer loaded
   Total parameters: 1.10B


## 5️⃣ LoRA 설정

In [ ]:
print("🔧 Setting up LoRA...")

# LoRA 설정
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=TARGET_MODULES,
)

# LoRA 적용
model = get_peft_model(model, lora_config)

# 학습 파라미터 확인
print("\n📊 Trainable Parameters:")
model.print_trainable_parameters()

# 계산
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
trainable_percentage = 100 * trainable_params / total_params

print(f"\n💡 LoRA는 전체 모델의 {trainable_percentage:.3f}%만 학습합니다!")
print(f"   전체: {total_params:,} 파라미터")
print(f"   학습: {trainable_params:,} 파라미터")

🔧 Setting up LoRA...

📊 Trainable Parameters:
trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079

💡 LoRA는 전체 모델의 0.727%만 학습합니다!
   전체: 620,111,872 파라미터
   학습: 4,505,600 파라미터


## 6️⃣ Fine-tuning 전 테스트 (Before)

In [ ]:
print("🧪 Testing ORIGINAL model (Before fine-tuning)\n")
print("=" * 70)

# 테스트 프롬프트
test_prompts = [
    "한국의 수도는 어디인가요?",
    "김치찌개 만드는 법을 알려주세요.",
    "파이썬으로 리스트를 정렬하는 방법은?",
]

# 원본 모델 로드 (LoRA 없이)
original_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16,
)
original_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if original_tokenizer.pad_token is None:
    original_tokenizer.pad_token = original_tokenizer.eos_token

# 파이프라인
pipe = pipeline(
    "text-generation",
    model=original_model,
    tokenizer=original_tokenizer,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)

# 테스트
for i, prompt in enumerate(test_prompts, 1):
    formatted_prompt = f"<|user|>\n{prompt}</s>\n<|assistant|>\n"
    result = pipe(formatted_prompt)[0]["generated_text"]

    # 답변 추출
    if "<|assistant|>" in result:
        answer = result.split("<|assistant|>")[-1].split("</s>")[0].strip()
    else:
        answer = result[len(formatted_prompt):].strip()

    print(f"\n❓ Test {i}: {prompt}")
    print(f"💬 Answer: {answer[:200]}...")
    print("-" * 70)

# 메모리 정리
del original_model, pipe
torch.cuda.empty_cache()

print("\n💡 대부분 영어로 답변하거나 이상한 답변을 하는 것을 확인할 수 있습니다.")
print("   Fine-tuning 후 어떻게 변하는지 지켜봐주세요!\n")

🧪 Testing ORIGINAL model (Before fine-tuning)


❓ Test 1: 한국의 수도는 어디인가요?
💬 Answer: Korea's capital city is Seoul, located in the southeastern part of the country....
----------------------------------------------------------------------

❓ Test 2: 김치찌개 만드는 법을 알려주세요.
💬 Answer: Kimchi is a traditional Korean dish made from fermented cabbage, garlic, and spices. Here's a simple recipe for making Kimchi at home:

Ingredients:
- 2 cups of fresh, finely chopped cabbage
- 2 garli...
----------------------------------------------------------------------

❓ Test 3: 파이썬으로 리스트를 정렬하는 방법은?
💬 Answer: Sure, here are a few methods for sorting a Python list in ascending or descending order:

1. Using built-in sort function:

```python
# Example 1: Using built-in sort function
sorted_list = [5, 3, 1, ...
----------------------------------------------------------------------

💡 대부분 영어로 답변하거나 이상한 답변을 하는 것을 확인할 수 있습니다.
   Fine-tuning 후 어떻게 변하는지 지켜봐주세요!



## 7️⃣ LoRA Fine-tuning 🚀

이제 본격적으로 학습을 시작합니다! (약 15-20분 소요)

In [ ]:
print("🚀 Starting LoRA Fine-tuning...\n")

# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
    warmup_steps=50,
    lr_scheduler_type="cosine",
    optim="paged_adamw_32bit" if USE_4BIT else "adamw_torch",
    gradient_checkpointing=True,
    report_to="none",
    group_by_length=True,
)

# Trainer 설정
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=MAX_SEQ_LENGTH,
    peft_config=lora_config,
)

# 학습 시작 시간 기록
start_time = datetime.now()
print(f"⏰ Training started at: {start_time.strftime('%H:%M:%S')}")
print(f"📊 Total steps: {len(trainer.get_train_dataloader()) * NUM_EPOCHS}")
print(f"\n{'='*70}\n")

# 학습 실행!
trainer.train()

# 학습 종료 시간
end_time = datetime.now()
duration = end_time - start_time
print(f"\n{'='*70}")
print(f"✅ Training completed!")
print(f"⏰ Duration: {duration}")
print(f"{'='*70}\n")

🚀 Starting LoRA Fine-tuning...

⏰ Training started at: 16:08:55


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override

📊 Total steps: 2235




/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.189700
20,1.161100
30,1.145700
40,1.199500
50,1.221800
60,1.099700
70,1.109800
80,1.079700
90,1.110000
100,1.086700


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a n


✅ Training completed!
⏰ Duration: 0:36:52.880041



In [ ]:
# LoRA adapter 저장
print(f"💾 Saving LoRA adapter to {OUTPUT_DIR}...")

trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Saved!")
print(f"\n📁 Saved files:")
!ls -lh {OUTPUT_DIR}

💾 Saving LoRA adapter to ./korean-tinyllama-lora...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Saved!

📁 Saved files:
total 20M
-rw-r--r-- 1 root root  688 Oct 16 16:45 adapter_config.json
-rw-r--r-- 1 root root  18M Oct 16 16:45 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K Oct 16 16:45 README.md
-rw-r--r-- 1 root root  551 Oct 16 16:45 special_tokens_map.json
-rw-r--r-- 1 root root 1.4K Oct 16 16:45 tokenizer_config.json
-rw-r--r-- 1 root root 1.8M Oct 16 16:45 tokenizer.json
-rw-r--r-- 1 root root 489K Oct 16 16:45 tokenizer.model


## 8️⃣ Fine-tuning 후 테스트 (After)

In [ ]:
print("🤖 Loading fine-tuned model...\n")

# Fine-tuned 모델 로드
finetuned_model = AutoPeftModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    device_map="auto",
    torch_dtype=torch.float16,
)
finetuned_model = finetuned_model.merge_and_unload()

finetuned_tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
if finetuned_tokenizer.pad_token is None:
    finetuned_tokenizer.pad_token = finetuned_tokenizer.eos_token

print("✅ Model loaded!")

🤖 Loading fine-tuned model...

✅ Model loaded!


In [ ]:
print("🧪 Testing FINE-TUNED model (After fine-tuning)\n")
print("=" * 70)

# 동일한 테스트 프롬프트
test_prompts = [
    "한국의 수도는 어디인가요?",
    "김치찌개 만드는 법을 알려주세요.",
    "파이썬으로 리스트를 정렬하는 방법은?",
]

# 파이프라인
pipe = pipeline(
    "text-generation",
    model=finetuned_model,
    tokenizer=finetuned_tokenizer,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)

# 테스트
for i, prompt in enumerate(test_prompts, 1):
    formatted_prompt = f"<|user|>\n{prompt}</s>\n<|assistant|>\n"
    result = pipe(formatted_prompt)[0]["generated_text"]

    # 답변 추출
    if "<|assistant|>" in result:
        answer = result.split("<|assistant|>")[-1].split("</s>")[0].strip()
    else:
        answer = result[len(formatted_prompt):].strip()

    print(f"\n❓ Test {i}: {prompt}")
    print(f"💬 Answer:\n{answer}")
    print("-" * 70)

print("\n✨ 이제 한국어로 자연스럽게 답변하는 것을 확인할 수 있습니다!")

🧪 Testing FINE-TUNED model (After fine-tuning)


❓ Test 1: 한국의 수도는 어디인가요?
💬 Answer:
한국의 수도는 서울, 출장샵, 노원, 홍콩 등입니다. 최근 발전된 수도는 노원이 되었습니다. 그리고 북쪽에 있는 홍콩도 더 빠르게 발전하고 있습니다. 한국의 수도는 약 60개입니다. 본문에서 알아본 수도는 홍콩, 출장샵, ��
----------------------------------------------------------------------

❓ Test 2: 김치찌개 만드는 법을 알려주세요.
💬 Answer:
김치찌개를 만드는 법을 설명하는 방법은 다음과 같습니다:

1. 잘려된 카페 베디드 플라스틱 블레이드에 알맞은 카페 블레이드 배달 매체를 넣습니다.

2. 크래스터 또는 카페 블레이드 배달 매체에 ��������
----------------------------------------------------------------------

❓ Test 3: 파이썬으로 리스트를 정렬하는 방법은?
💬 Answer:
리스트를 정렬하는 방법에는 여러 가지 방법이 있습니다. 다음은 파이썬 3.8 버전에서 가장 일반적인 방법입니다:

```python
# 원본 리스트에서 요소 0부터 25까지 순회하고 그 값을 리스트의 배열에 추가합니다.
arr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
arr.sort()
print(arr)
----------------------------------------------------------------------

✨ 이제 한국어로 자연스럽게 답변하는 것을 확인할 수 있습니다!


In [ ]:
print("🧪 Original vs Fine-tuned Model Comparison\n")
print("=" * 70)

# 원본 모델 로드 (이미 로드되어 있다면 생략 가능)
try:
    original_model
except NameError:
    print("🤖 Loading original model and tokenizer for comparison...")
    original_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float16,
    )
    original_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    if original_tokenizer.pad_token is None:
        original_tokenizer.pad_token = original_tokenizer.eos_token
    print("✅ Original model loaded!")


# 원본 모델 파이프라인
original_pipe = pipeline(
    "text-generation",
    model=original_model,
    tokenizer=original_tokenizer,
    max_new_tokens=150, # 원본 테스트 셀과 동일하게 설정
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)

# Fine-tuned 모델 파이프라인 (이미 로드되어 있다면 재사용)
# 'pipe' 변수는 이전 셀에서 정의된 finetuned_model 파이프라인입니다.
try:
    pipe # 이전 셀에서 정의된 finetuned_model 파이프라인 사용
except NameError:
    print("🤖 Loading fine-tuned model for comparison...")
    finetuned_model = AutoPeftModelForCausalLM.from_pretrained(
        OUTPUT_DIR,
        device_map="auto",
        torch_dtype=torch.float16,
    )
    finetuned_model = finetuned_model.merge_and_unload()
    finetuned_tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
    if finetuned_tokenizer.pad_token is None:
        finetuned_tokenizer.pad_token = finetuned_tokenizer.eos_token

    pipe = pipeline(
        "text-generation",
        model=finetuned_model,
        tokenizer=finetuned_tokenizer,
        max_new_tokens=200, # fine-tuned 테스트 셀과 동일하게 설정
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )
    print("✅ Fine-tuned model loaded!")


# 비교할 질문 (이전 셀에서 정의된 your_question 변수 사용)
try:
    your_question
except NameError:
    your_question = "대한민국의 수도는 어디인가요?" # your_question이 정의되지 않았다면 기본값 사용
    print(f"⚠️ 'your_question'이 정의되지 않아 기본 질문 '{your_question}'을 사용합니다.")


formatted_prompt = f"<|user|>\n{your_question}</s>\n<|assistant|>\n"

print(f"❓ Your Question: {your_question}\n")

# 원본 모델 응답
print("=== Original Model Response ===")
original_result = original_pipe(formatted_prompt)[0]["generated_text"]
if "<|assistant|>" in original_result:
    original_answer = original_result.split("<|assistant|>")[-1].split("</s>")[0].strip()
else:
    original_answer = original_result[len(formatted_prompt):].strip()
print(original_answer)
print("-" * 30)

# Fine-tuned 모델 응답
print("=== Fine-tuned Model Response ===")
finetuned_result = pipe(formatted_prompt)[0]["generated_text"]
if "<|assistant|>" in finetuned_result:
    finetuned_answer = finetuned_result.split("<|assistant|>")[-1].split("</s>")[0].strip()
else:
    finetuned_answer = finetuned_result[len(formatted_prompt):].strip()
print(finetuned_answer)
print("-" * 30)

print("\n✨ 두 모델의 답변 차이를 확인해보세요!")

# 메모리 정리 (선택사항)
# del original_model, original_pipe
# torch.cuda.empty_cache()

🧪 Original vs Fine-tuned Model Comparison

🤖 Loading original model and tokenizer for comparison...
✅ Original model loaded!
⚠️ 'your_question'이 정의되지 않아 기본 질문 '대한민국의 수도는 어디인가요?'을 사용합니다.
❓ Your Question: 대한민국의 수도는 어디인가요?

=== Original Model Response ===
대한민국의 수도는 대한민국 전체 사이트에 있습니다. 대한민국에 있는 모든 도시, 군사 기관, 공공 기관, 정부 설립체계 등이 모두 포함되어 있습니다.
------------------------------
=== Fine-tuned Model Response ===
대한민국의 주요 수도는 홍콩, 뉴욕, 캘리포니아, 베르베이니치, 루노리아, 엘리베이터, 말레인바, 포르토네, 카탈랜드, 크래디언, 카르토크릭, 브라질, 베이비 코리아, 아메리칸, 네덜란드, 베네수엘라,
------------------------------

✨ 두 모델의 답변 차이를 확인해보세요!


## 9️⃣ 직접 테스트해보기!

In [ ]:
# 원하는 질문을 입력해보세요!
your_question = "대한민국의 전통 음식에는 무엇이 있나요?"  # 여기를 수정하세요

print(f"❓ Your Question: {your_question}\n")
print("=" * 70)

formatted_prompt = f"<|user|>\n{your_question}</s>\n<|assistant|>\n"
result = pipe(formatted_prompt)[0]["generated_text"]

# 답변 추출
if "<|assistant|>" in result:
    answer = result.split("<|assistant|>")[-1].split("</s>")[0].strip()
else:
    answer = result[len(formatted_prompt):].strip()

print(f"💬 Answer:\n{answer}")
print("=" * 70)

❓ Your Question: 대한민국의 전통 음식에는 무엇이 있나요?

💬 Answer:
대한민국의 전통 음식에는 다음과 같은 여러 종류가 있습니다:

1. 농업식: 농업식은 농업 산물을 통해 생산되는 질소 소진의 초반 결과물입니다. 농업식은 당연히 농업에 따라 달라집니다.

2. 치마: 치마는 농업에 따라 달라�


## 🔟 모델 다운로드 (선택사항)

In [ ]:
# LoRA adapter를 zip으로 압축하여 다운로드
!zip -r korean-tinyllama-lora.zip {OUTPUT_DIR}

print("✅ Zipped!")
print("📥 다운로드: 좌측 파일 탭에서 'korean-tinyllama-lora.zip'을 찾아 다운로드하세요.")
print("\n💡 로컬에서 사용하는 방법:")
print("""\nfrom peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

# LoRA adapter 로드
model = AutoPeftModelForCausalLM.from_pretrained("./korean-tinyllama-lora")
model = model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained("./korean-tinyllama-lora")

# 추론
prompt = "<|user|>\\n안녕하세요!</s>\\n<|assistant|>\\n"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
""")

updating: korean-tinyllama-lora/ (stored 0%)
updating: korean-tinyllama-lora/adapter_model.safetensors (deflated 8%)
updating: korean-tinyllama-lora/README.md (deflated 66%)
updating: korean-tinyllama-lora/tokenizer.json (deflated 74%)
updating: korean-tinyllama-lora/tokenizer_config.json (deflated 69%)
updating: korean-tinyllama-lora/special_tokens_map.json (deflated 79%)
updating: korean-tinyllama-lora/tokenizer.model (deflated 55%)
updating: korean-tinyllama-lora/adapter_config.json (deflated 52%)
✅ Zipped!
📥 다운로드: 좌측 파일 탭에서 'korean-tinyllama-lora.zip'을 찾아 다운로드하세요.

💡 로컬에서 사용하는 방법:

from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

# LoRA adapter 로드
model = AutoPeftModelForCausalLM.from_pretrained("./korean-tinyllama-lora")
model = model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained("./korean-tinyllama-lora")

# 추론
prompt = "<|user|>\n안녕하세요!</s>\n<|assistant|>\n"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(*

## 1️⃣1️⃣ Google Drive에 저장 (선택사항)

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# Drive에 복사
!cp -r {OUTPUT_DIR} /content/drive/MyDrive/korean-tinyllama-lora

print("✅ Saved to Google Drive!")
print("📁 Location: MyDrive/korean-tinyllama-lora")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Saved to Google Drive!
📁 Location: MyDrive/korean-tinyllama-lora


## 📊 요약 및 다음 단계

### ✅ 완료한 작업
1. ✅ TinyLlama 모델 로드
2. ✅ 한국어 데이터셋 준비
3. ✅ LoRA 설정 및 적용
4. ✅ Fine-tuning 실행
5. ✅ Before/After 비교 테스트

### 🎯 주요 결과
- **전체 파라미터**: 1.1B
- **학습 파라미터**: ~5-10M (약 0.5%만 학습!)
- **메모리 사용량**: 4-bit 사용 시 ~6GB
- **학습 시간**: T4 GPU 기준 ~15-20분

### 🚀 다음 단계
1. **더 많은 데이터로 학습**: `NUM_SAMPLES`를 늘려보세요
2. **하이퍼파라미터 튜닝**: `LORA_R`, `LEARNING_RATE` 조정
3. **다른 모델 시도**: Llama-2, Mistral 등
4. **DPO (Direct Preference Optimization)**: 선호도 기반 추가 학습
5. **배포**: HuggingFace Hub에 업로드하여 공유

### 📚 참고 자료
- [LoRA 논문](https://arxiv.org/abs/2106.09685)
- [PEFT 라이브러리](https://github.com/huggingface/peft)
- [TRL 라이브러리](https://github.com/huggingface/trl)

---

### 🎉 축하합니다!
LoRA를 사용하여 영어 모델을 한국어로 성공적으로 적응시켰습니다!